In [ ]:
import sys
sys.path.append('..')
import umbrella_mesh
import elastic_rods
import linkage_vis
from umbrella_mesh import UmbrellaEnergyType
from elastic_rods import EnergyType
from bending_validation import suppress_stdout as so

In [ ]:
import numpy as np

In [ ]:
input_path = '../../data/sphere_cap_0.3.json'
from load_jsondata import read_data
input_data, io = read_data(filepath = input_path)
width = 2*input_data['arm_plate_edge_offset']
thickness = width * 0.5 # 1.5 mm # FIX from mm to meters everywhere
cross_section = [thickness, width]

### Initialization

In [ ]:
curr_um = umbrella_mesh.UmbrellaMesh(io)
curr_um.setMaterial(elastic_rods.RodMaterial('rectangle', 2000, 0.3, cross_section, stiffAxis=elastic_rods.StiffAxis.D1))
curr_um.targetDeploymentHeight = thickness * 1

In [ ]:
curr_um.energy(UmbrellaEnergyType.Elastic)

In [ ]:
curr_um.set_design_parameter_config(True, False)

In [ ]:
# curr_um.gradient(variableDesignParameters = True, designParameterOnly = False)

### Parameters

In [ ]:
curr_um.uniformDeploymentEnergyWeight = 2
curr_um.targetDeploymentHeight = thickness * 1

In [ ]:
view = linkage_vis.LinkageViewer(curr_um, width=1024, height=600)
view.averagedMaterialFrames = True
view.show()

In [ ]:
curr_um.hessian(variableDesignParameters = True)

### Hessian Validation

In [ ]:
curr_um.updateSourceFrame()

In [ ]:
curr_um.hessianSparsityPattern()

In [ ]:
curr_um.hessian()

In [ ]:
import umbrella_finite_diff

In [ ]:
import importlib
importlib.reload(umbrella_finite_diff)

In [ ]:
perturbation = np.random.uniform(-0.1, 0.1, curr_um.numDoF())

In [ ]:
dof = curr_um.getDoFs()

In [ ]:
import copy

In [ ]:
curr_um.setDoFs(dof + perturbation)
# curr_um.setDoFs(perturbation_along_segment_dof)

In [ ]:
view.update()

In [ ]:
curr_um.uniformDeploymentEnergyWeight = 200

In [ ]:
curr_um.targetDeploymentHeight = thickness

In [ ]:
curr_um.updateSourceFrame()

In [ ]:
umbrella_finite_diff.hessian_convergence_plot(curr_um, umbrellaEnergyType=umbrella_mesh.UmbrellaEnergyType.Deployment, etype=elastic_rods.EnergyType.Full, maxStepSize=1e-1, variableDesignParameters=True, perSegmentRestLen=True)

In [ ]:
direction = np.random.uniform(-1, 1, curr_um.numExtendedDoFPSRL())

In [ ]:
umbrella_finite_diff.hessian_convergence_plot(curr_um, umbrellaEnergyType=umbrella_mesh.UmbrellaEnergyType.Elastic, etype=elastic_rods.EnergyType.Full, variableDesignParameters=True, direction = direction, perSegmentRestLen=True)

In [ ]:
umbrella_finite_diff.hessian_convergence_plot(curr_um, umbrellaEnergyType=umbrella_mesh.UmbrellaEnergyType.Elastic, etype=elastic_rods.EnergyType.Full, variableDesignParameters=False, direction = direction[:curr_um.numDoF()])

In [ ]:
var_types = ['FULL', 'RodSegment', 'Joint']
var_indices = {'FULL': range(0, curr_um.numDoF()),
               'RodSegment': range(0, curr_um.dofOffsetForJoint(0)),
               'Joint': range( curr_um.dofOffsetForJoint(0), curr_um.numDoF())}

In [ ]:
umbrella_finite_diff.hessian_convergence_block_plot(curr_um, var_types, var_indices, umbrellaEnergyType=umbrella_mesh.UmbrellaEnergyType.Full, etype=elastic_rods.EnergyType.Full, variableDesignParameters=True, perSegmentRestLen=True)

In [ ]:
position_indices = [np.arange(curr_um.dofOffsetForJoint(i), curr_um.dofOffsetForJoint(i) + 3) for i in range(curr_um.numJoints())]
position_indices = np.array(position_indices).flatten()

omega_indices = [np.arange(curr_um.dofOffsetForJoint(i) + 3, curr_um.dofOffsetForJoint(i) + 6) for i in range(curr_um.numJoints())]
omega_indices = np.array(omega_indices).flatten()

alpha_indices = [np.arange(curr_um.dofOffsetForJoint(i) + 6, curr_um.dofOffsetForJoint(i) + 7) for i in range(curr_um.numJoints())]
alpha_indices = np.array(alpha_indices).flatten()

length_indices = []
for i in range(curr_um.numJoints()):
    length_indices += list(np.arange(curr_um.dofOffsetForJoint(i) + 7, curr_um.dofOffsetForJoint(i) + curr_um.joint(i).numDoF()))
length_indices = np.array(length_indices)

In [ ]:
var_types = ['position', 'omega', 'alpha', 'length']
var_indices = {'position': position_indices,
               'omega': omega_indices,
               'alpha': alpha_indices,
               'length': length_indices}

In [ ]:
umbrella_finite_diff.hessian_convergence_block_plot(curr_um, var_types, var_indices, umbrellaEnergyType=umbrella_mesh.UmbrellaEnergyType.Full, etype=elastic_rods.EnergyType.Full, perSegmentRestLen=True)